In [1]:
#coding:utf-8

# 必要なモジュールをインポート
import codecs
import corenlp
import csv
import nltk
import pandas as pd
import pydot
import numpy as np
import re
import subprocess
from stemming.porter2 import stem
import xml.etree.ElementTree as ET

### Anaconda環境に新しくインストールするもの
#### Starnford Corenlp
基本はここ => http://qiita.com/yubessy/items/1869ac2c66f4e76cd6c5
<br>ただし、pipを使う時に、anaconda環境のpipでインストールする。
例えば、以下のようなコマンド。
```
.pyenv/versions/anaconda2-2.5.0/bin/pip install corenlp-python
```
#### stem
ここからダウンロード => https://pypi.python.org/pypi/stemming/1.0
<br>ディレクトリに行って、以下のコマンド。
```
python setupy.py install
```
#### pydot
以下のコマンドで入る。
```
conda install -c anaconda pydot=1.0.28
```

In [2]:
# まずデータをダウンロード
f = open("./nlp.txt","rb")
nlp_data = f.read()

In [3]:
nlp_data

'Natural language processing\nFrom Wikipedia, the free encyclopedia\n\nNatural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages. As such, NLP is related to the area of humani-computer interaction. Many challenges in NLP involve natural language understanding, that is, enabling computers to derive meaning from human or natural language input, and others involve natural language generation.\n\nHistory\n\nThe history of NLP generally starts in the 1950s, although work can be found from earlier periods. In 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence.\n\nThe Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sentences into English. The authors claimed that within three or five years, machine translati

## 50. 文区切り
(. or ; or : or ? or !) → 空白文字 → 英大文字というパターンを文の区切りと見なし，入力された文書を1行1文の形式で出力せよ．

#### [方針]
正規表現を使ってパターン分割する。

In [4]:
pattern = r'\.\s[A-Z]|\;\s[A-Z]|\:\s[A-Z]|\?\s[A-Z]|\!\s[A-Z]'
sentences = re.finditer(pattern, nlp_data)

In [7]:
index_list=[]
sentence_list = []
for i,so in enumerate(sentences):
	if i == 0:
		_0_index = so.end()-1
		sentence_list.extend([nlp_data[:_0_index].rstrip()])

		print nlp_data[:_0_index].rstrip()
		index_list.extend([_0_index])

	else:
		start_index = index_list[i-1]
		end_index = so.end()-1
		sentence_list.extend([nlp_data[start_index:end_index]])

		print nlp_data[start_index:end_index]
		index_list.extend([end_index])

Natural language processing
From Wikipedia, the free encyclopedia

Natural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages.
As such, NLP is related to the area of humani-computer interaction. 
Many challenges in NLP involve natural language understanding, that is, enabling computers to derive meaning from human or natural language input, and others involve natural language generation.

History

The history of NLP generally starts in the 1950s, although work can be found from earlier periods. 
In 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence.

The Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sentences into English. 
The authors claimed that within three or five years, machine translation woul

#### [解説]
(参照) http://kenichia.hatenablog.com/entry/2016/02/15/174813

* 正規表現の意味
    ```
    pattern = r'\.\s[A-Z]|\;\s[A-Z]|\:\s[A-Z]|\?\s[A-Z]|\!\s[A-Z]'
    ```
    \ => ここから始まり<br>
	\s => スペース(空白文字)<br>
	[A-Z] => 英大文字<br>
	| => または<br>

	つまり、".などの記号+スペース+英大文字"の組み合わせをorで4種類作っているということ<br>
	若干これは冗長な気がする。記号の部分をひとつにまとめられたらきっともっとすっきりするはず。


* 正規表現で文字列を探すとき
	(参照)http://qiita.com/wanwanland/items/ce272419dde2f95cdabc
		- re.split()は、patternを削除して分割する
		- re.match/search/findall/finditer とかの使い分けが大事

## 51. 単語の切り出し
	空白を単語の区切りとみなし，50の出力を入力として受け取り，1行1単語の形式で出力せよ．ただし，文の終端では空行を出力せよ．

In [11]:
word_list = []
for sentence in sentence_list:
	for word in re.split(" ", sentence):     # 空白で分割
		if word[:-1] == ".":                 # インデックスの"-1"は、一番後ろを指す
			print word,"\n"
			eos_word = word+"\n"
			word_list.extend([eos_word])
		else:
			print word
			word_list.extend([word])

Natural
language
processing
From
Wikipedia,
the
free
encyclopedia

Natural
language
processing
(NLP)
is
a
field
of
computer
science,
artificial
intelligence,
and
linguistics
concerned
with
the
interactions
between
computers
and
human
(natural)
languages.
As
such,
NLP
is
related
to
the
area
of
humani-computer
interaction.

Many
challenges
in
NLP
involve
natural
language
understanding,
that
is,
enabling
computers
to
derive
meaning
from
human
or
natural
language
input,
and
others
involve
natural
language
generation.

History

The
history
of
NLP
generally
starts
in
the
1950s,
although
work
can
be
found
from
earlier
periods.

In
1950,
Alan
Turing
published
an
article
titled
"Computing
Machinery
and
Intelligence"
which
proposed
what
is
now
called
the
Turing
test
as
a
criterion
of
intelligence.

The
Georgetown
experiment
in
1954
involved
fully
automatic
translation
of
more
than
sixty
Russian
sentences
into
English.

The
authors
claimed
that
within
three
or
five
years,
machine
translation
woul

## 52. ステミング
	51の出力を入力として受け取り，Porterのステミングアルゴリズムを適用し，単語と語幹をタブ区切り形式で出力せよ．
    Pythonでは，Porterのステミングアルゴリズムの実装としてstemmingモジュールを利用するとよい．

* stemのインストール =>
(参照) https://pypi.python.org/pypi/stemming/1.0
<br> ダウンロードして、"python setup.py install"を叩く！！

In [12]:
for word in word_list:
	print word,"\t",stem(word)

Natural 	Natur
language 	languag
processing
From 	processing
From
Wikipedia, 	Wikipedia,
the 	the
free 	free
encyclopedia

Natural 	encyclopedia

Natur
language 	languag
processing 	process
(NLP) 	(NLP)
is 	is
a 	a
field 	field
of 	of
computer 	comput
science, 	science,
artificial 	artifici
intelligence, 	intelligence,
and 	and
linguistics 	linguist
concerned 	concern
with 	with
the 	the
interactions 	interact
between 	between
computers 	comput
and 	and
human 	human
(natural) 	(natural)
languages. 	languages.
As 	As
such, 	such,
NLP 	NLP
is 	is
related 	relat
to 	to
the 	the
area 	area
of 	of
humani-computer 	humani-comput
interaction. 	interaction.
 	
Many 	Mani
challenges 	challeng
in 	in
NLP 	NLP
involve 	involv
natural 	natur
language 	languag
understanding, 	understanding,
that 	that
is, 	is,
enabling 	enabl
computers 	comput
to 	to
derive 	deriv
meaning 	mean
from 	from
human 	human
or 	or
natural 	natur
language 	languag
input, 	input,
and 	and
others 	other
involve 	involv
natu

## 53. Tokenization
Stanford Core NLPを用い，入力テキストの解析結果をXML形式で得よ．また，このXMLファイルを読み込み，入力テキストを1行1単語の形式で出力せよ．

* 方針<br>
    corenlpはコマンドライン上で動かした方が早い気がするので、以下のjavaをコンパイルすると、自動でxmlファイルができます。(1分未満)
    <br>ただし、-file "input.txt"の部分を、インプットするファイル名に変える。
```
	java -cp "*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit,pos,lemma,ner,parse,dcoref -file input.txt
```
   
* 注意<br>
    corenlpのダウンロードするversionが古いと、エラー等で動かないことがある<br>
    (参照) http://qiita.com/yubessy/items/1869ac2c66f4e76cd6c5
	


In [13]:
xml_file_name = "./nlp.txt.xml"
tree = ET.parse(xml_file_name)
for word in tree.iter('word'):
	print word.text

Natural
language
processing
From
Wikipedia
,
the
free
encyclopedia
Natural
language
processing
-LRB-
NLP
-RRB-
is
a
field
of
computer
science
,
artificial
intelligence
,
and
linguistics
concerned
with
the
interactions
between
computers
and
human
-LRB-
natural
-RRB-
languages
.
As
such
,
NLP
is
related
to
the
area
of
humani-computer
interaction
.
Many
challenges
in
NLP
involve
natural
language
understanding
,
that
is
,
enabling
computers
to
derive
meaning
from
human
or
natural
language
input
,
and
others
involve
natural
language
generation
.
History
The
history
of
NLP
generally
starts
in
the
1950s
,
although
work
can
be
found
from
earlier
periods
.
In
1950
,
Alan
Turing
published
an
article
titled
``
Computing
Machinery
and
Intelligence
''
which
proposed
what
is
now
called
the
Turing
test
as
a
criterion
of
intelligence
.
The
Georgetown
experiment
in
1954
involved
fully
automatic
translation
of
more
than
sixty
Russian
sentences
into
English
.
The
authors
claimed
that
within
three
or
five

* その他<br>
    pythonで"subprocess"というモジュールを使うと、コマンドライン上での操作をpythonのコード内に挟むこともできる

## 54. 品詞タグ付け
	Stanford Core NLPの解析結果XMLを読み込み，単語，レンマ，品詞をタブ区切り形式で出力せよ．
	(参照) http://qiita.com/segavvy/items/4d55805352089332828e

In [14]:
for token in tree.iter("token"):
	print token.findtext("word"),"\t",token.findtext("lemma"),"\t",token.findtext("POS")

Natural 	natural 	JJ
language 	language 	NN
processing 	processing 	NN
From 	from 	IN
Wikipedia 	Wikipedia 	NNP
, 	, 	,
the 	the 	DT
free 	free 	JJ
encyclopedia 	encyclopedia 	NN
Natural 	natural 	JJ
language 	language 	NN
processing 	processing 	NN
-LRB- 	-lrb- 	-LRB-
NLP 	nlp 	NN
-RRB- 	-rrb- 	-RRB-
is 	be 	VBZ
a 	a 	DT
field 	field 	NN
of 	of 	IN
computer 	computer 	NN
science 	science 	NN
, 	, 	,
artificial 	artificial 	JJ
intelligence 	intelligence 	NN
, 	, 	,
and 	and 	CC
linguistics 	linguistics 	NNS
concerned 	concern 	VBN
with 	with 	IN
the 	the 	DT
interactions 	interaction 	NNS
between 	between 	IN
computers 	computer 	NNS
and 	and 	CC
human 	human 	JJ
-LRB- 	-lrb- 	-LRB-
natural 	natural 	JJ
-RRB- 	-rrb- 	-RRB-
languages 	language 	NNS
. 	. 	.
As 	as 	IN
such 	such 	JJ
, 	, 	,
NLP 	nlp 	NN
is 	be 	VBZ
related 	relate 	VBN
to 	to 	TO
the 	the 	DT
area 	area 	NN
of 	of 	IN
humani-computer 	humani-computer 	JJ
interaction 	interaction 	NN
. 	. 	.
Many 	many 	JJ
challenges 	cha

## 	55. 固有表現抽出
	入力文中の人名をすべて抜き出せ．
* NER => Named Entity Recognition

* 方針<br>
	NERのタグに含まれているタグは以下の8種類。<br>
	{'DATE', 'DURATION', 'LOCATION', 'MISC', 'NUMBER', 'ORDINAL', 'ORGANIZATION', 'PERSON'}<br>
	このうち人名に当たるタグは"PERSON"なので、これを抜き出す。<br>
	ところで、"MISC"ってなんですかね！調べたけどわからなかったです。<br>

(参考) http://qiita.com/segavvy/items/32b3a35825ec32586f33

In [16]:
nerList=[]
for token in tree.iter("token"):
	if token.findtext("NER") == "PERSON":
		nerList.extend([token.findtext("word")])
print set(nerList)

set(['Weizenbaum', 'Moore', 'Jabberwacky', 'Schank', 'Turing', 'Lehnert', 'Alan', 'Joseph', 'Carbonell', 'Meehan', 'MARGIE', 'Wilensky'])


## 	56. 共参照解析
	Stanford Core NLPの共参照解析の結果に基づき，文中の参照表現（mention）を代表参照表現（representative mention）に置換せよ．ただし，置換するときは，「代表参照表現（参照表現）」のように，元の参照表現が分かるように配慮せよ．



* 解説<br>
	指示語を元の語に復元することがタスクとなっている。<br>
	xmlファイルの構造は2つ => "sentences"と"coreference"

	=> "coreference"の部分をいじくる。<br>
	例えば...
    ```
	<coreference>
	<mention representative="true">		=> 参照元となっている、元の単語・フレーズ
          <sentence>20</sentence>		=> 20文目の
          <start>17</start>				=> 17単語目から始まり
          <end>25</end>					=> 25単語目で終わる
          <head>19</head>				=> 係り受けでこのフレーズで核をなす単語は19単語目。今回なら"rules"
          <text>hard if-then rules similar to existing hand-written rules</text>
        </mention>
        <mention>						=> 二つ目以降のmentionが、一つ目のmentionを参照している
          <sentence>20</sentence>		=> 具体的には、20文目の7~8単語目にある"algorithms"
          <start>7</start>
          <end>8</end>
          <head>7</head>
          <text>algorithms</text>
        </mention>
      </coreference>					=> ここで参照関係の記述終わり。
    ```
こんな感じの中身。

	=> 具体的に行う処理は、mention(2つ目以降)にある単語を特定して、そこにmention representative="true"を代入する、という作業になる。

<br>
* 参考<br> 
http://kenichia.hatenablog.com/entry/2016/02/15/192635 <br>
http://qiita.com/segavvy/items/0340d3d71c9151265bcb


In [23]:
# 56について以下は、ここからほぼ借用しています => http://qiita.com/segavvy/items/0340d3d71c9151265bcb
root = tree.getroot()

# 修正に必要な情報をまとめる。キーは参照元の名詞句、値は参照先の名詞句のアドレスを何らかの形で。
modify_dic={}
for coreference in root.iterfind('./document/coreference/coreference'):

	# 代表参照表現の取得
	refered_text = coreference.findtext('./mention[@representative="true"]/text')

	# 代表参照表現以外のmention列挙、辞書に追加
	for mention in coreference.iterfind('./mention'):
		if mention.get('representative', 'false') == 'false':

		# 必要な情報の抽出
			sent_id = int(mention.findtext('sentence'))
			start_index = int(mention.findtext('start'))
			end_index = int(mention.findtext('end'))

			if not (sent_id, start_index) in modify_dic:
				modify_dic[(sent_id, start_index)] = (end_index, refered_text)

	# ここから修正しながら表示する
for sentence in root.iterfind('./document/sentences/sentence'):
	sent_id = int(sentence.get('id'))       # sentenceのid
	org_rest = 0                            # 置換中のtoken数の残り
	string_array = ""

	# token列挙
	for token in sentence.iterfind('./tokens/token'):
		token_id = int(token.get('id'))     # tokenのid

		# 置換対象？
		if org_rest == 0 and (sent_id, token_id) in modify_dic:

           	# 辞書から終了位置と代表参照表現を取り出し
			(end, rep_text) = modify_dic[(sent_id, token_id)]

			# 代表参照表現＋カッコを挿入
			string_array = string_array + '[' + refered_text + ']' + " "
			# print '[' + refered_text + ']'
			org_rest = end - token_id       # 置換中のtoken数の残り

		# token出力
		string_array = string_array + token.findtext('word') +" "
		# print token.findtext('word')

	   # 置換の終わりなら閉じカッコを挿入
		if org_rest > 0:
			org_rest -= 1
		# if org_rest == 0:
			# string_array = string_array + ')'
			# print(')')

	print string_array

Natural language processing From Wikipedia , the free encyclopedia Natural language processing -LRB- NLP -RRB- is [Natural Language Learning -LRB- NLL -RRB-] a field of computer science , artificial intelligence , and linguistics concerned with the interactions between computers and human -LRB- natural -RRB- languages . 
As such , NLP is related to the area of humani-computer interaction . 
Many challenges in NLP involve natural language understanding , that is , enabling [Natural Language Learning -LRB- NLL -RRB-] computers to derive meaning from human or natural language input , and others involve natural language generation . 
History The history of NLP generally starts in the 1950s , although work can be found from earlier periods . 
In 1950 , Alan Turing published an article titled `` Computing Machinery and Intelligence '' which proposed what is now called the [Natural Language Learning -LRB- NLL -RRB-] Turing test as a criterion of intelligence . 
The Georgetown experiment in 19

## 	57. 係り受け解析
	Stanford Core NLPの係り受け解析の結果（collapsed-dependencies）を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

* AnacondaにてGraphvizインストール
```
conda install -c anaconda graphviz=2.38.0
```

* 処理のイメージ
<br>具体的には、ROOTと各単語、依存関係の発生している方向を各エッジについて読み込んで、それをネットワークとして描画。

In [25]:
# ひとまず、依存関係を各文章毎に書き出してみる
edge_list=[]
for i,sentence in enumerate(root[0][0]):
	print i,"番目の文の依存関係は..."
	tmp=[]
	for dependency in sentence[3]:
		print dependency[0].text,"->",dependency[1].text
		tmp.append(tuple([dependency[0].text,dependency[1].text]))
	edge_list.append(tmp)
	print "\n"

0 番目の文の依存関係は...
ROOT -> field
processing -> Natural
processing -> language
field -> processing
processing -> From
processing -> Wikipedia
processing -> the
processing -> free
processing -> encyclopedia
processing -> Natural
processing -> language
field -> processing
processing -> NLP
field -> is
field -> a
science -> of
science -> computer
field -> science
intelligence -> artificial
field -> intelligence
field -> and
field -> linguistics
linguistics -> concerned
interactions -> with
interactions -> the
concerned -> interactions
computers -> between
interactions -> computers
computers -> and
languages -> human
languages -> natural
computers -> languages


1 番目の文の依存関係は...
ROOT -> related
such -> As
related -> such
related -> NLP
related -> is
area -> to
area -> the
related -> area
interaction -> of
interaction -> humani-computer
area -> interaction


2 番目の文の依存関係は...
ROOT -> involve
challenges -> Many
involve -> challenges
NLP -> in
challenges -> NLP
understanding -> natural
understanding

In [26]:
# 同じディレクトリ以下にgraph_pngという書き出し用のディレクトリを作成して、そこにグラフを書き出す。
for i,edges in enumerate(edge_list):
	g=pydot.graph_from_edges(edges, directed=True)
	file_name = "./graph_png/nlp_"+str(i)+".png"
	g.write_png(file_name, prog='dot')

## 58. タプルの抽出
	Stanford Core NLPの係り受け解析の結果（collapsed-dependencies）に基づき，「主語 述語 目的語」の組をタブ区切り形式で出力せよ．
    ただし，主語，述語，目的語の定義は以下を参考にせよ．

    述語: nsubj関係とdobj関係の子（dependant）を持つ単語
    主語: 述語からnsubj関係にある子（dependent）
    目的語: 述語からdobj関係にある子（dependent）

* 方針<br>
	各文に対して、述語をfixして、それに紐づく主語と目的語を抽出する


* (参照) 	
<br>http://qiita.com/segavvy/items/f100fc38e350ad14b679
<br>		http://kenichia.hatenablog.com/entry/2016/02/15/192635


In [28]:
for i,sentence in enumerate(root[0][0]):
	for dep in sentence[3]:

		if "nsubj" in dep.get("type"):
			nsubj_govnr = dep.find('./governor').text
			nsubj_dep = dep.find('./dependent').text

		if "dobj" in dep.get("type"):
			dobj_govnr = dep.find('./governor').text
			dobj_dep = dep.find('./dependent').text

			if nsubj_govnr == dobj_govnr:
				print nsubj_dep,"\t",dobj_govnr,"\t",dobj_dep

others 	involve 	generation
Turing 	published 	article
what 	called 	test
experiment 	involved 	translation
ELIZA 	provided 	interaction
patient 	exceeded 	base
ELIZA 	provide 	response
which 	structured 	information
underpinnings 	discouraged 	sort
that 	underlies 	approach
Some 	produced 	systems
which 	make 	decisions
that 	contains 	errors
implementations 	involved 	coding
algorithms 	take 	set
Some 	produced 	systems
which 	make 	decisions
models 	have 	advantage
they 	express 	certainty
Systems 	have 	advantages
Automatic 	make 	use
that 	make 	decisions


## 	59. S式の解析
	Stanford Core NLPの句構造解析の結果（S式）を読み込み，文中のすべての名詞句（NP）を表示せよ．入れ子になっている名詞句もすべて表示すること．
    
* 方針<br>
	S式の中身はこのようになっている。
   ```
<parse>(ROOT (S (ADVP (RB Increasingly)) (, ,) (ADVP (RB however)) (, ,) (NP (NN research)) (VP (VBZ has) (VP (VBN focused) (PP (IN on) (NP (NP (JJ statistical) (NNS models)) (, ,) (SBAR (WHNP (WDT which)) (S (VP (VBP make) (NP (JJ soft) (, ,) (JJ probabilistic) (NNS decisions)) (PP (VBN based) (PP (IN on) (S (VP (VBG attaching) (NP (JJ real-valued) (NNS weights)) (PP (TO to) (NP (DT each) (NN input) (NN feature)))))))))))))) (. .))) </parse>
```
この式の中からNPを取ってくる。正規表現を適当に使う。<br>
NPのすぐ左の(から、任意の数の()セットを終えて、)が得られるまでを取ってくる。

In [29]:
NP_list=[]
for i,sentence in enumerate(root[0][0]):

	_S = sentence.find("./parse").text,"\n"
	tmp_list = re.findall(r"(\(NP\s(\((JJ|NN|NNS)\s\w+\)\s*)(\((JJ|NN|NNS)\s\w+\)\s*)+\))" ,_S[0])
	if not len(tmp_list) ==0:
		NP_list.extend(tmp_list)

for i in NP_list:
	print i

('(NP (JJ Natural) (NN language) (NN processing))', '(JJ Natural) ', 'JJ', '(NN processing)', 'NN')
('(NP (NN computer) (NN science))', '(NN computer) ', 'NN', '(NN science)', 'NN')
('(NP (JJ artificial) (NN intelligence))', '(JJ artificial) ', 'JJ', '(NN intelligence)', 'NN')
('(NP (JJ Many) (NNS challenges))', '(JJ Many) ', 'JJ', '(NNS challenges)', 'NNS')
('(NP (JJ natural) (NN language) (NN understanding))', '(JJ natural) ', 'JJ', '(NN understanding)', 'NN')
('(NP (JJ natural) (NN language) (NN generation))', '(JJ natural) ', 'JJ', '(NN generation)', 'NN')
('(NP (NN machine) (NN translation))', '(NN machine) ', 'NN', '(NN translation)', 'NN')
('(NP (JJ real) (NN progress))', '(JJ real) ', 'JJ', '(NN progress)', 'NN')
('(NP (JJ Little) (JJ further) (NN research))', '(JJ Little) ', 'JJ', '(NN research)', 'NN')
('(NP (NN machine) (NN translation))', '(NN machine) ', 'NN', '(NN translation)', 'NN')
('(NP (NN translation) (NNS systems))', '(NN translation) ', 'NN', '(NNS systems)', 'NNS